In [1]:
import tensorflow as tf
import json
import numpy as np
import os
from PIL import Image

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
names = json.load(open('/run/media/backman/yay/ui_layout_vectors/ui_names.json'))

In [3]:
names = names['ui_names']
names = np.asarray(names)

In [4]:
len(names)

72103

In [5]:
vectors = np.load('/run/media/backman/yay/ui_layout_vectors/ui_vectors.npy')

In [6]:
vectors.shape

(72103, 64)

In [7]:
idxs = np.random.permutation(len(names))

In [8]:
max = 125*125

In [9]:
vectors = vectors[idxs][:max]

In [10]:
size, embedding_size = vectors.shape

In [11]:
part = names[idxs][:max]


In [12]:
p2i = dict((v,k) for k,v in enumerate(part))

In [13]:
part.shape, vectors.shape

((15625,), (15625, 64))

In [14]:
indexes = max

In [15]:
tf.reset_default_graph()

In [16]:
embeddings = tf.get_variable('embedding', initializer = tf.constant(vectors))

In [17]:
indexes = tf.get_variable('index', initializer = tf.constant(indexes))

In [18]:
embed = tf.nn.embedding_lookup(embeddings, indexes)

In [19]:
metadata_file = '/run/media/backman/yay/ui_embs/metadata.tsv'
config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
emb_proj = config.embeddings.add()
emb_proj.tensor_name = 'embedding'
emb_proj.sprite.image_path = 'spritesheet.png'
emb_proj.sprite.single_image_dim.extend([64,64])
emb_proj.metadata_path = metadata_file
summary_writer = tf.summary.FileWriter('/run/media/backman/yay/ui_embs/')
tf.contrib.tensorboard.plugins.projector.visualize_embeddings(summary_writer, config)

In [20]:
root_dir = '/run/media/backman/yay/unique_uis/resized/'
# images = [Image.open(root_dir + filename.replace('png','jpg')) for filename in part]

In [21]:
width = 64 * 125
height = 64 * 125

In [22]:
master = Image.new(
mode = 'RGBA',
size = (width, height),
color=(255,255,255,0)
)

In [23]:
metadata = 'Name\tId\n'
for i,image in enumerate(part):
    location_x = (i % 125) * 64 + 14
    location_y = (i // 125) * 64
    location = (location_x, location_y)
    master.paste(Image.open(root_dir + image.replace('png','jpg')), location)
    metadata += f'{image}\t{i}\n'
    
master.save('/run/media/backman/yay/ui_embs/spritesheet.png')
m_file = open("/run/media/backman/yay/ui_embs/metadata.tsv","w")
m_file.write(metadata)
m_file.close()

In [24]:
saver = tf.train.Saver()
session = tf.Session()
session.run(tf.global_variables_initializer())
saver.save(session, os.path.join('/run/media/backman/yay/ui_embs/', "model.ckpt"), 1)

'/run/media/backman/yay/ui_embs/model.ckpt-1'

In [40]:
close_to_me = tf.placeholder(tf.float32, [1, vectors.shape[1]])

norm_emb = tf.nn.l2_normalize(embeddings, dim=1)
norm_ele = tf.nn.l2_normalize(close_to_me, dim=1)

cosine_sim = tf.matmul(norm_ele, tf.transpose(norm_emb, [1,0]))

closest_ui = tf.argmax(cosine_sim, 1)

top_uis = tf.nn.top_k(cosine_sim, k=10)


In [41]:
res = session.run(top_uis, feed_dict = {close_to_me: vectors[9270:9271,:]})

In [44]:
print(res.indices)

[[  534  2724  3304  3500  7638  9270  9876 10988 11979 13456]]
